## Project 6- Pipeline 2 - Search

In [1]:
import lib.database_module as db
import lib.encoding_module as en
from sklearn.externals import joblib
#transformer = joblib.load('data/vectorizer.pkl')

In [2]:
search_string = 'bubbles'

In [3]:
sql = """SELECT page, page_id
         FROM page 
         """
page_texts = db.execute_sql_statement(sql,'local')
clean_pages = [x[0] for x in page_texts]
page_ids = [x[1] for x in page_texts]

OK


I'll just build a new transformer, since I have many more rows than the remote server.

In [ ]:
transformer = en.build_vectorizer(clean_pages)
joblib.dump(transformer, 'data/vectorizer2.pkl')

In [4]:
transformer = joblib.load('data/vectorizer2.pkl')

### Parse: Encode search as document vector	

In [5]:
search_string = 'bubbles'
search_vector = en.get_searchterm_vector(transformer,search_string)
search_vector

{'bubbles': array([[  7.72128980e-03,  -3.12928160e-03,   3.24774443e-03,
          -2.48504148e-03,   7.61812091e-04,   3.20403832e-03,
           2.12430610e-03,  -2.01797367e-03,  -5.50662039e-03,
           2.52223710e-03,   1.97770309e-03,   6.41191556e-03,
           4.18997397e-03,  -5.01587926e-03,  -4.45226175e-03,
           6.42394659e-04,  -1.13002251e-03,  -4.54448908e-03,
           5.09831999e-03,   6.46433290e-03,  -4.85265043e-03,
          -6.39291927e-03,   4.13683638e-03,  -1.44077783e-02,
           8.18552683e-03,   2.20390998e-02,  -1.11878927e-02,
           7.72058248e-03,  -1.30406979e-02,   2.49483772e-02,
          -3.25296141e-02,   1.65893698e-02,  -5.23169293e-03,
           9.72143904e-03,  -4.72252584e-03,  -9.26355962e-03,
          -2.43885005e-02,   8.69155924e-03,  -1.32012249e-02,
          -5.52551584e-02,  -8.95138360e-02,  -1.37898772e-04,
           4.81034778e-02,   7.52231762e-02,   8.68758818e-02,
           3.46251959e-02,   8.74966587e-03,

#### create the vectors for page vector table.

In [6]:
len(page_ids), len(clean_pages)

(4583, 4583)

In [7]:
import lib.encoding_funcs as ef

In [ ]:
#page_vecs = ef.get_page_vector(transformer, clean_pages[:50], page_ids[:50])

In [9]:
page_texts[0]

('A sports car (sportscar) is a small, usually two seater, two door automobile designed for spirited performance and nimble handling. According to the Merriam-Webster dictionary, the first known use of the term was in 1928.\nSports cars may be spartan or luxurious, but high maneuverability and minimum weight are requisite. They may be equipped for racing, especially an aerodynamically shaped one-passenger or two-passenger vehicle having a low center of gravity and steering and suspension designed for precise control at high speeds.\n\nEarly history\nThe definition of a sports car is not precise, but from the earliest first automobiles people have found ways to make them go faster, round corners better, and look more beautiful than the ordinary models inspiring an emotional relationship with a car that is fun to drive and use for the sake of driving. The basis for the sports car is traced to the early 20th century touring cars and roadsters. These raced in early rallies, such as the Her

In [ ]:
%%timeit -n 1 -r 1
connection, cursor = db.connect_to_postgres('local')
cursor.execute('DELETE FROM page_vec;')
connection.commit()

for text, page_id in page_texts:
    vector  = list(transformer.transform([text])[0])
    sql = """INSERT INTO page_vec (page_id, page_vector)
          VALUES ({}, ARRAY{})
          on conflict do nothing;""".format(page_id, vector)
    #print sql
    cursor.execute(sql)
    connection.commit()
cursor.close()
connection.close()    


In [ ]:
connection, cursor = db.connect_to_postgres('local')
cursor.execute('DELETE FROM page_vec;')
connection.commit()
for page_id, page_vector in page_vecs.items():
    sql = """INSERT INTO page_vec (page_id, page_vector)
          VALUES ({}, ARRAY{})""".format(page_id, list(page_vector[0]))
    #print sql
    temp1 = cursor.execute(sql)
    temp2= connection.commit()
                     
cursor.close()
connection.close()

In [ ]:
temp1, temp2

### Mine: Select document vectors for all pages from database

In [ ]:
pages_vectors = db.select_all_page_vectors(location='local')


In [ ]:
len(pages_vectors)

### Model: Find five most similar documents based on document vectors

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:



page_match_list = []

### Mine: Retrieve text for the five most similar documents

In [ ]:

match_results = db.select_pages(page_match_list,'local')

### BONUS: Present: Command is executed via command line script	